# Optimize an Electrode

In [ ]:
recon_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/Richardson_DBS/Reco_Chron_Pain/VCVS_VPL/ea_reconstruction.mat'
target_path = '/Users/cu135/Partners HealthCare Dropbox/Calvin Howard/studies/collaborations/Richardson_DBS/optimized_program/target_maps/flipped_SEEG_pain_intensity_vs_sham_inverted_pcc_inverse_r_map.nii'
out_dir = '/Users/cu135/hires_backdrops/test_electrode'

In [ ]:
from stim_pyper.processing_utils.run_optimizer import OptimizerProcessor
optimizer = OptimizerProcessor(electrode_data=recon_path, nifti_path=target_path, output_path=out_dir)
d = optimizer.run()